In [ ]:
%%capture
!pip install openai &> /dev/null
!pip install gradio

In [ ]:
import gradio as gr
import openai

# There is no need for an API key let the following be as is
openai.api_key = "EMPTY"

openai.api_base = "http://zanino.millennium.berkeley.edu:8000/v1"
# If there is any issue try using
# openai.api_base = "http://34.132.127.197:8000/v1"

def get_gorilla_response(prompt, model="gorilla-7b-hf-v1"):
  completion = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content": prompt}]
  )
  return completion.choices[0].message.content

def parse_output(text):
  components = {}

  components['domain'] = text.split("<<<domain>>>:")[1].split("<<<api_call>>>")[0].strip()
  components['api_call'] = text.split("<<<api_call>>>:")[1].split("<<<api_provider>>>")[0].strip()
  components['api_provider'] = text.split("<<<api_provider>>>:")[1].split("<<<explanation>>>")[0].strip()
  components['explanation'] = text.split("<<<explanation>>>:")[1].split("<<<code>>>")[0].strip()
  components['code'] = text.split("<<<code>>>:")[1].strip()

  return components

def parse_and_display(prompt):

  text = get_gorilla_response(prompt)

  components = parse_output(text)

  domain = components['domain']
  api_call = components['api_call']
  api_provider = components['api_provider']
  explanation = components['explanation']
  code = components['code']

  return domain, api_call, api_provider, explanation, code

examples = [
  ["Translate this sentence to French"],
  ["Summarize this paragraph in 5 sentences"],
  ["Generate an image of a cat"]
]

iface = gr.Interface(fn=parse_and_display,
                     inputs="text",
                     outputs=[
                       gr.components.Textbox(label="Domain"),
                       gr.components.Textbox(label="API Call"),
                       gr.components.Textbox(label="API Provider"),
                       gr.components.Textbox(label="Explanation"),
                       gr.components.Code(label="Code")
                     ],
                     title="Gorilla API Explorer",
                     description="Empower Your Interactions with Gorilla API Through an Intuitive and User-Friendly Interface",
                     examples=examples)

iface.launch(share=True)